In [1]:
import utils.age_processing as ap

In [2]:
import pandas as pd
import os
from dotenv import load_dotenv
from pathlib import Path

env_path = Path('../..') / 'config.env'
load_dotenv(dotenv_path=env_path)
dataset_dir = os.getenv("dataset_directory")



In [3]:
dataset = 'bx' # ml, mlhd, or bx
year = 2013 # Only for mlhd dataset
filtered = True 
weighted = True
age_type = 'finegrained_age'

In [ ]:
if dataset == 'ml':    
    data_dir = dataset_dir + f'/processed/ml_rec{"_filtered" if filtered else ""}'
    user_stats_path = dataset_dir + f'/processed/movielens-1m'
elif dataset == 'mlhd':
    data_dir = dataset_dir + f'/processed/mlhd_rec{"_filtered" if filtered else ""}_{year}'
    user_stats_path = dataset_dir + f'/processed/mlhd_sampled_filtered'
elif dataset == 'bx':
    data_dir = dataset_dir + f'/processed/bx_rec{"_filtered" if filtered else ""}'
    user_stats_path = dataset_dir + f'/processed/Book-Crossing'
    

In [10]:
user_info_path = data_dir + '/user_info.tsv'
user_stats = pd.read_csv(user_stats_path + f'/user_profile_stats{"_weighted" if weighted else ""}.tsv', sep='\t')
user_info = pd.read_csv(user_info_path, sep='\t')

ages_sort = ap.get_sorted_ages(dataset, age_type)

FileNotFoundError: [Errno 2] No such file or directory

In [ ]:
user_stats['age_group'] = user_stats['age'].apply(lambda x: ap.age_group(x, dataset, age_type))
user_stats['age_group'] = pd.Categorical(user_stats['age_group'], categories=ages_sort, ordered=True)

user_info['age_group'] = user_info['age'].apply(lambda x: ap.age_group(x, dataset, age_type))
user_info['age_group'] = pd.Categorical(user_info['age_group'], categories=ages_sort, ordered=True)


In [ ]:
filtered_stats_ratio = filtered_users['age_at_listen'].value_counts(normalize=True)


In [ ]:
user_stats